In [4]:
import pandas as pd
from sqlalchemy import create_engine

In [1]:
# Clase para realizar consultas básicas a una base de datos y sus tablas
class BasicQueries:
    def __init__(self, credentials):
        """
        Inicializa la clase con un diccionario de credenciales y establece la conexión.
        """
        self.isConnected = False
        self.credentials = credentials

        # Construcción de la cadena de conexión a partir del diccionario de credenciales
        self.engstr = (
            f"{self.credentials['servidor']}://"
            f"{self.credentials['user']}:{self.credentials['user_pwd']}@"
            f"{self.credentials['IP/localhost']}/{self.credentials['db']}"
        )
        self.connect()  # instanciar un método propio de la clase que estamos escribiendo

    def connect(self):
        """
        Establece la conexión a la base de datos usando SQLAlchemy.
        """
        try:
            self.engine = create_engine(self.engstr)
            self.conn = self.engine.raw_connection()
            self.isConnected = True
        except Exception as e:
            print(f"No se pudo conectar a la base de datos. Error: {e}")

    def consultar_tablas(self):
        """
        Consulta las tablas disponibles en la base de datos.
        """
        return pd.read_sql(f"SHOW TABLES FROM {self.credentials['db']}", self.conn)

    def consultar_esquema(self, tabla):
        """
        Consulta el esquema de una tabla específica: nombres de columnas, tipo de datos y si permite nulos.
        """
        return pd.read_sql(
            f"""
            SELECT column_name, data_type, is_nullable
            FROM information_schema.columns
            WHERE table_name = '{tabla}'
            """,
            self.conn
        )

    def consultar_primeras_filas(self, tabla):
        """
        Consulta las primeras 5 filas de una tabla.
        """
        return pd.read_sql(f"SELECT * FROM {tabla} LIMIT 5", self.conn)

    def query(self, query):
        """
        Ejecuta cualquier consulta SQL y devuelve el resultado como DataFrame.
        """
        return pd.read_sql(query, self.conn)


In [2]:
credentials = {
    "servidor": "mysql+pymysql",
    "user": "root",
    "user_pwd": "Yeiko1208",
    "IP/localhost": "localhost",
    "db": "banco_base"
}

In [5]:
# instanciamos la clase con las credenciales
Q = BasicQueries(credentials)

In [7]:
#Realizamos las consultas con el objeto que instancia el engine
pd.read_sql("Select * from tarjetas",Q.conn)

/var/folders/p5/32ttydq52jq48h56ch0qt6d40000gn/T/ipykernel_9469/3554551409.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("Select * from tarjetas",Q.conn)


,id_tarjeta,tipo,cliente,monto
0,1,Credito,1,126210
1,2,Credito,2,259351
2,3,Debito,3,15184
3,4,Debito,4,218341
4,5,Debito,5,479564
...,...,...,...,...
95,96,Debito,96,246300
96,97,Credito,97,81937
97,98,Debito,98,120466
98,99,Debito,99,474459


In [ ]:
# 1. Lista los nombres y edades de todos los clientes que tengan ingresos mayores a 50,000.
query = """
        SELECT nombre, edad
        FROM clientes
        WHERE ingresos > 50000;
        """
df = pd.read_sql(query, Q.conn)
display(df)

In [ ]:
# 2. Obtén el número total de clientes por país, ordenado de mayor a menor.
query = """
        SELECT pais, COUNT(nombre) AS numero_total_clientes
        FROM clientes
        GROUP BY pais
        ORDER BY COUNT(nombre) DESC;
        """
df = pd.read_sql(query, Q.conn)
display(df)

In [ ]:
# 3. Muestra los clientes (id_cliente, nombre, ingresos) cuyo estrato sea 2 y que además tengan al menos una tarjeta asociada.
query = """
        SELECT id_cliente, nombre, ingresos
        FROM clientes
        JOIN tarjetas ON clientes.id_cliente = tarjetas.cliente
        WHERE estrato = 2;
        """
df = pd.read_sql(query, Q.conn)
display(df)